# Programming Assignment: Рекомендательные системы

## Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок «Смотрели ранее» — в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корелляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать `recall@k` и `precision@k`.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке — например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

## Данные

Вам дается две выборки с пользовательскими сессиями – `id`-шниками просмотренных и `id`-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: `id` просмотренных товаров через `,` затем идёт `;`, после чего следуют `id` купленных товаров (если такие имеются), разделённые запятой. Например, `1,2,3,4;` или `1,2,3,4;5,6`.

Гарантируется, что среди `id` купленных товаров все различные.

**Важно:**
* Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
* Если товар не встречался в обучающей выборке, его популярность равна 0.
* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и `k` в `recall@k`/`precision@k`.


## Задание

In [2]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from collections import Counter

In [3]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [4]:
def str2list(string):
    if len(string) == 0:
        return None
    try: 
        return np.array(string.split(','),dtype=np.uint32)
    except ValueError as e:
        print('string: ', string)

In [5]:
class OrderedCounter(Counter, OrderedDict):
    'Counter that remembers the order elements are first encountered'

    def __repr__(self):
        return '%s(%r)' % (self.__class__.__name__, OrderedDict(self))

    def __reduce__(self):
        return self.__class__, (OrderedDict(self),)

### Задание 1. 

На обучении постройте частоты появления `id` в просмотренных и в купленных (`id` может несколько раз появляться в просмотренных, все появления надо учитывать).

In [45]:
data_train = pd.read_table("coursera_sessions_train.txt",sep=';',header=None,converters={0:str2list, 1:str2list})
data_test = pd.read_table("coursera_sessions_test.txt",sep=';',header=None,converters={0:str2list, 1:str2list})

data_train.columns=['see','buy']
data_test.columns=['see','buy']

In [7]:
data_train.head(10)

,see,buy
0,"[0, 1, 2, 3, 4, 5]",None
1,"[9, 10, 11, 9, 11, 12, 9, 11]",None
2,"[16, 17, 18, 19, 20, 21]",None
3,"[24, 25, 26, 27, 24]",None
4,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",None
5,[42],None
6,"[47, 48, 49]",None
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]"
8,"[71, 72, 73, 74]",None
9,"[76, 77, 78]",None


In [8]:
data_train.buy.dropna().shape

(3608,)

* создаем список купленных товаров по сессиям, получится приблизительно такое [[0,1,2,3],[15,16],[4]...]
* создаем словарь просмотренных товаров (ключ - id товара, значение - его частота) как вариант, создать пустой словарь, затем перебором списка ВСЕХ просмотренных товаров добавить в словарь, если их нет, если они повторяются -- инкрементировать значение

In [9]:
buy_list = data_train.buy.dropna().values
ordered_c = OrderedCounter(np.concatenate(data_train.see.values))

* создать список, состоящий из списков просмотренных товаров за каждую сессию) ПОСЛЕ dropna

In [10]:
see_list_dropna = data_train.dropna().see

In [1]:
a=[1, 0, 0, 1]
b=[1, 1, 0, 0] 

In [21]:
(1+1+2.0/3+2.0/4)*0.5-(1+1.0/2+1.0/3+2.0/4)*0.5

0.41666666666666674

* двойным перебором полученного в предыдущем пункте списка преобразовать его в список кортежей вида [[(60,2) ,(59,3)], [(130,1)]...]. частоту брать из словаря. следует обратить внимание на то, что получится список списков кортежей типа (х,у)

In [11]:
pd.unique([5,5,6,4,57,12,5])

array([ 5,  6,  4, 57, 12])

In [12]:
set([5,5,6,4,57,12,5])

{4, 5, 6, 12, 57}

In [13]:
list_of_pair = []
for list_ in see_list_dropna:
    lst = []
    for elem_ in pd.unique(list_):
        lst.append((elem_,ordered_c[elem_]))
    list_of_pair.append(lst)

* отсортировать кортежи в каждом списке полученного массива

In [14]:
len(list_of_pair)

3608

In [15]:
see_list_dropna.values

array([array([59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 68, 67], dtype=uint32),
       array([84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86], dtype=uint32),
       array([138, 198, 199, 127], dtype=uint32), ...,
       array([64552, 25931,  2807], dtype=uint32),
       array([91921, 20251,  5063, 21742,  5063, 20251, 34927], dtype=uint32),
       array([32291, 60520, 32291, 38220], dtype=uint32)], dtype=object)

In [16]:
for tb in list_of_pair:
    tb.sort(key=lambda x: x[1],reverse=True)

* обратное преобразование в список просмотренных товаров по сессиям (берется только нулевая компонента кортежа (x,y)[0])

In [17]:
sorted_list = []
for list_ in list_of_pair:
    sorted_list.append(map(lambda x: x[0],list_))

### Задание 2. 

Реализуйте два алгоритма рекомендаций: сортировка просмотренных `id` по популярности (частота появления в просмотренных), сортировка просморенных `id` по покупаемости (частота появления в покупках). Если частота одинаковая, то сортировать надо по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

In [18]:
# def Precision(session, dic_sort,dic_set , k):
#     see=session.see
    
#     buy = session.buy
    
#     k = min(k,see.shape[0])
#     see = f7(see)
    
#     see = [s for s in see if s in dic_set]
    
#     if len(see) == 0:
#         return 0
    
#     sorted(see,key= lambda x: train_see_dic[x],reverse=True)
#     reccomend = see[:k]
    
#     bough_rec = np.intersect1d(reccomend,buy)
    
#     return bough_rec.shape[0]/float(k)

In [31]:
def Precision(see,buy, k):
#     
#     k = min(k,len(see))
    
#     bough_rec = np.intersect1d(see[:k],buy)
    bough_rec = [s for s in buy if s in see[:k]]
    
    return len(bough_rec)/float(k)

In [ ]:
# def Recall(session, buy,dic_set, k):
#     see=session.see
    
#     buy = session.buy
#     k = min(k,see.shape[0])
#     t = min(see.shape[0],buy.shape[0])
#     see = f7(see)
    
    
    
#     see = [s for s in see if s in dic_set]
    
#     if len(see) == 0:
#         return 0
    
#     sorted(see,key= lambda x: train_see_dic[x],reverse=True)
#     reccomend = see[:k]
    
#     bough_rec = np.intersect1d(reccomend,buy)
    
#     return bough_rec.shape[0]/float(t)

In [32]:
def Recall(see,buy, k):

    k = min(k,len(see))
    t = min(len(see),len(buy))
#     k = min(k,t)
    
#     see = [s for s in see if s in dic_set]
    
    bough_rec = [s for s in buy if s in see[:k]]#np.intersect1d(see[:k],buy)
    
    return len(bough_rec)/float(t)

### Задание 3. 

Для данных алгоритмов выпишите через пробел `AverageRecall@1`, `AveragePrecision@1`, `AverageRecall@5`, `AveragePrecision@5` на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

In [22]:
len_test = len(sorted_list)

In [23]:
len_test

3608

In [33]:
precision_buy_test_1 = 0.0
precision_buy_test_5 = 0.0

recall_buy_test_1 = 0.0
recall_buy_test_5 = 0.0
for buy,see in zip(buy_list,sorted_list):
    precision_buy_test_1 += Precision(see,buy,1)
    precision_buy_test_5 += Precision(see,buy,5)
    recall_buy_test_1 += Recall(see,buy,1)
    recall_buy_test_5 += Recall(see,buy,5)
print recall_buy_test_1/len_test,precision_buy_test_1/len_test,\
recall_buy_test_5/len_test,precision_buy_test_5/len_test

0.44601416845 0.512195121951 0.82959512771 0.212527716186


In [92]:
[0.4706289814521518, 0.5454545454545454, 0.8162751297729124, 0.2093680709534424]

[0.4706289814521518,
 0.5454545454545454,
 0.8162751297729124,
 0.2093680709534424]

In [93]:
ans1=[recall_buy_test_1/len_test,precision_buy_test_1/len_test,\
recall_buy_test_5/len_test,precision_buy_test_5/len_test]
print ans1

[0.4460141684504208, 0.5121951219512195, 0.8295951277104273, 0.21252771618625918]


In [94]:
ans1=map(lambda x : round(x,2),ans1)

In [95]:
str(ans1)

'[0.45, 0.51, 0.83, 0.21]'

In [96]:
with open("ans1","w") as f:
    for an in ans1:
        f.write(str(an)+' ')

### Тестовая выборка

In [46]:
buy_list = data_test.buy.dropna().values
see_list_dropna = data_test.dropna().see
list_of_pair = []
for list_ in see_list_dropna:
    lst = []
    for elem_ in pd.unique(list_):
        lst.append((elem_,ordered_c[elem_]))
    list_of_pair.append(lst)
    
for tb in list_of_pair:
    tb.sort(key=lambda x: x[1],reverse=True)
    
    
sorted_list = []
for list_ in list_of_pair:
    sorted_list.append(map(lambda x: x[0],list_))

In [47]:
len_test = len(sorted_list)
print len_test

3665


In [48]:
precision_buy_test_1 = 0.0
precision_buy_test_5 = 0.0

recall_buy_test_1 = 0.0
recall_buy_test_5 = 0.0
for buy,see in zip(buy_list,sorted_list):
    precision_buy_test_1 += Precision(see,buy,1)
    precision_buy_test_5 += Precision(see,buy,5)
    recall_buy_test_1 += Recall(see,buy,1)
    recall_buy_test_5 += Recall(see,buy,5)
print recall_buy_test_1/len_test,precision_buy_test_1/len_test,\
recall_buy_test_5/len_test,precision_buy_test_5/len_test

0.421215595834 0.481309686221 0.804883057361 0.203765347885


In [49]:
ans2=[recall_buy_test_1/len_test,precision_buy_test_1/len_test,\
recall_buy_test_5/len_test,precision_buy_test_5/len_test]
print ans2
ans2=map(lambda x : round(x,2),ans2)
with open("ans2","w") as f:
    for an in ans2:
        f.write(str(an)+' ')

[0.4212155958336038, 0.48130968622100956, 0.8048830573611896, 0.2037653478854079]


## Обучение на просмотрах

In [54]:
ordered_c = OrderedCounter(np.concatenate(data_train.buy.dropna().values))

buy_list = data_train.buy.dropna().values
see_list_dropna = data_train.dropna().see
list_of_pair = []
for list_ in see_list_dropna:
    lst = []
    for elem_ in pd.unique(list_):
        lst.append((elem_,ordered_c[elem_]))
    list_of_pair.append(lst)
    
for tb in list_of_pair:
    tb.sort(key=lambda x: x[1],reverse=True)
    
    
sorted_list = []
for list_ in list_of_pair:
    sorted_list.append(map(lambda x: x[0],list_))

len_test = len(sorted_list)
print len_test

precision_buy_test_1 = 0.0
precision_buy_test_5 = 0.0

recall_buy_test_1 = 0.0
recall_buy_test_5 = 0.0
for buy,see in zip(buy_list,sorted_list):
    precision_buy_test_1 += Precision(see,buy,1)
    precision_buy_test_5 += Precision(see,buy,5)
    recall_buy_test_1 += Recall(see,buy,1)
    recall_buy_test_5 += Recall(see,buy,5)
print recall_buy_test_1/len_test,precision_buy_test_1/len_test,\
recall_buy_test_5/len_test,precision_buy_test_5/len_test

In [59]:
ans3=[recall_buy_test_1/len_test,precision_buy_test_1/len_test,\
recall_buy_test_5/len_test,precision_buy_test_5/len_test]
print ans3
ans3=map(lambda x : round(x,2),ans2)
with open("ans3","w") as f:
    for an in ans2:
        f.write(str(an)+' ')

[0.6922932604473627, 0.8037694013303769, 0.9312244635138202, 0.2525498891352649]


### тестовая выборка

In [60]:
ordered_c = OrderedCounter(np.concatenate(data_train.buy.dropna().values))

buy_list = data_test.buy.dropna().values
see_list_dropna = data_test.dropna().see
list_of_pair = []
for list_ in see_list_dropna:
    lst = []
    for elem_ in pd.unique(list_):
        lst.append((elem_,ordered_c[elem_]))
    list_of_pair.append(lst)
    
for tb in list_of_pair:
    tb.sort(key=lambda x: x[1],reverse=True)
    
    
sorted_list = []
for list_ in list_of_pair:
    sorted_list.append(map(lambda x: x[0],list_))

len_test = len(sorted_list)
print len_test

precision_buy_test_1 = 0.0
precision_buy_test_5 = 0.0

recall_buy_test_1 = 0.0
recall_buy_test_5 = 0.0
for buy,see in zip(buy_list,sorted_list):
    precision_buy_test_1 += Precision(see,buy,1)
    precision_buy_test_5 += Precision(see,buy,5)
    recall_buy_test_1 += Recall(see,buy,1)
    recall_buy_test_5 += Recall(see,buy,5)
print recall_buy_test_1/len_test,precision_buy_test_1/len_test,\
recall_buy_test_5/len_test,precision_buy_test_5/len_test

3665
0.464368894443 0.527694406548 0.825042921216 0.210095497954


In [61]:
ans4=[recall_buy_test_1/len_test,precision_buy_test_1/len_test,\
recall_buy_test_5/len_test,precision_buy_test_5/len_test]
print ans4
ans4=map(lambda x : round(x,2),ans4)
with open("ans4","w") as f:
    for an in ans4:
        f.write(str(an)+' ')

[0.4643688944427246, 0.5276944065484311, 0.8250429212157806, 0.21009549795362173]


### Дополнительные вопросы

* Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров `recall@k` меняется. Подумайте, как оценить минимальное и максимальное значение `recall@k` в зависимости от правила сортировки.
* Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.